In [90]:
# !pip install pandas
# !pip install pyyaml
# !pip install hpo3

In [91]:
import os
import pandas as pd
import numpy as np
from pyhpo import Ontology, HPOSet, Omim, stats, Gene
# import yaml

Ontology(os.path.join(os.getcwd(), 'phenotype', 'rawdl_20240310'))

In [92]:
# Declare the folder path for phenotype data source
phenotype_folder = os.path.join(os.getcwd(), 'phenotype', 'subset')

# Declare 7 Files
icd10omim = 'icd102omim_subset.tsv'
loinc2hpo = 'loinc2hpo_standardized.tsv'
orpha2omim = 'orpha2omim_subset.tsv'
snomed2hpo = 'snomed2hpo_subset.tsv'
snomed2orpha = 'snomed2orpha_subset.tsv'

# iderare yaml configuration file
yaml_file = 'iderare.yaml'

# Clinical data dummy in txt format separated with new line
clinical_data = 'clinical_data.txt'

# Read the clinical data and parse the data
icd10omim_df = pd.read_csv(os.path.join(phenotype_folder, icd10omim), sep='\t')
loinc2hpo_df = pd.read_csv(os.path.join(phenotype_folder, loinc2hpo), sep='\t')
orpha2omim_df = pd.read_csv(os.path.join(phenotype_folder, orpha2omim), sep='\t')
snomed2hpo_df = pd.read_csv(os.path.join(phenotype_folder, snomed2hpo), sep='\t')
snomed2orpha_df = pd.read_csv(os.path.join(phenotype_folder, snomed2orpha), sep='\t')

# Read line from clinical_data and parse the data to list
with open(clinical_data, 'r') as file:
    clinical_data_list = file.read().splitlines()

In [93]:
# Convert SNOMED to ORPHA First
def snomed_orpha_parser(clinical_data, df):
    print('Trying to parse ORPHA from SNOMED-CT', clinical_data)
    
    if 'SNOMEDCT:' in clinical_data:
        if clinical_data not in df['code'].unique() :
            print('This SNOMED-CT code is not a clinical finding mapped with ORPHA, please check the SNOMED to ORPHA for diagnosis mapping.')
            snomed_sugg = df[df['code'].str.contains(clinical_data.strip('SNOMEDCT:'))]['code'].drop_duplicates()
            print('Sugggestion : It is possible that you mean any of this code:', (', ').join(snomed_sugg.values) , '?\n')
            return []
        else : 
            snomed_orpha = df[df['code'] == clinical_data]['orphanet_map'].to_list()
            print('Parsing of', clinical_data, 'successful with result of :', len(snomed_orpha), ' ORPHA code : ', (', ').join(snomed_orpha), '\n')
            return snomed_orpha
        
    else :
        print('The terminology is not recognized, please check if you have used correct terminology.')
        print('Allowable format : SNOMEDCT disorder semantic only SNOMEDCT:1212005', '\n')
        return []

In [94]:
# HPO Parser for Clinical Finding Related Terminology such as SNOMED, LOINC
def hpo_parser(clinical_data, df):    
    print('Trying to parse HPO from terminology', clinical_data)
    
    # LOINC case
    if 'LOINC:' in clinical_data:
        loinc_split = clinical_data.split('|')
        # Ensure that the forwarded data contain both LOINC and its interpretation
        if len(loinc_split) == 2:
            loinc = loinc_split[0]
            interpretation = loinc_split[1]
            
            # Handling if LOINC code not found in the database
            if loinc not in df['loinc'].unique() :
                print('LOINC data is not found in the database, please check the exact LOINC code.')
                loinc_sugg = df[df['loinc'].str.contains(loinc.strip('LOINC:'))]['loinc'].drop_duplicates()
                print('Did you mean any of this code:', (', ').join(loinc_sugg.values) , '?\n')
                return []
            
            loinc_hpo = df[df['loinc'] == loinc]

            # Handling if interpretation not suitable for the LOINC code
            if interpretation not in loinc_hpo['interpretation'].unique() :
                print('Interpretation is invalid, please check if you have used correct interpretation.')
                interpretation_sugg = df[df['loinc']==loinc]['interpretation'].drop_duplicates()
                print('Available interpretation for code', loinc, ' : ', (' or ').join(interpretation_sugg.values) , '\n')
                return []
            
            loinc_hpo = loinc_hpo[loinc_hpo['interpretation'] == interpretation]['hpoTermId'].to_list()
            print('Parsing of', clinical_data, 'successful with result of :', loinc_hpo, '\n')
            return list(loinc_hpo)
        else:
            print('LOINC data is missing either code / interpretation.')
            print('Example : LOINC:721-1|H for Qn lab examination OR LOINC:721-1|NEG for Nominal / Ordinal Lab Examination', '\n')
            return []
        
    # SNOMED-CT case
    elif 'SNOMEDCT:' in clinical_data:
        if clinical_data not in df['SNOMED_CT_ID'].unique() :
            print('This SNOMED-CT code is not a clinical finding mapped with HPO, please check the SNOMED to OMIM for diagnosis mapping.')
            snomed_sugg = df[df['SNOMED_CT_ID'].str.contains(clinical_data.strip('SNOMEDCT:'))]['SNOMED_CT_ID'].drop_duplicates()
            print('Sugggestion : It is possible that you mean any of this code:', (', ').join(snomed_sugg.values) , '?\n')
            return []
        else : 
            snomed_hpo = df[df['SNOMED_CT_ID'] == clinical_data]['HPO_ID'].to_list()
            print('Parsing of', clinical_data, 'successful with result of :', len(snomed_hpo), ' HPO code : ', (', ').join(snomed_hpo), '\n')
            return list(snomed_hpo)
        
    # Not recognized case
    else:
        print('The terminology is not recognized, please check if you have used correct terminology.')
        print('Example : LOINC:2862-1|L for Qn lab examination OR LOINC:725-2|NEG for categoric lab examination of SNOMEDCT:48610005 for Clinical Finding', '\n')
        return []

In [95]:
# OMIM Parser used for diagnosis related terminology ICD-10, ORPHA, SNOMED-CT to be translated to OMIM
def omim_parser(clinical_data,df):
    print('Trying to parse OMIM from terminology', clinical_data)

    # ICD-10 case
    if 'ICD-10:' in clinical_data:
        if clinical_data not in df['ICD10'].unique() :
            print('ICD-10 data is not found in the database, please check the exact ICD-10 code.')
            icd_sugg = df[df['ICD10'].str.contains(clinical_data.strip('ICD-10:'))]['ICD10'].drop_duplicates()
            print('Did you mean any of this code:', (', ').join(icd_sugg.values) , '?\n')
            return []
        else : 
            icd_omim = df[df['ICD10'] == clinical_data]['OMIM'].to_list()
            print('Parsing of', clinical_data, 'successful with result of :', len(icd_omim), ' OMIM code : ', (', ').join(icd_omim), '\n')
            return icd_omim
        
    # ORPHA case
    elif 'ORPHA:' in clinical_data:
        if clinical_data not in df['ORPHA'].unique() :
            print('ORPHA data is not found in the database, please check the exact ORPHA code.')
            orpha_sugg = df[df['ORPHA'].str.contains(clinical_data.strip('ORPHA:'))]['ORPHA'].drop_duplicates()
            print('Did you mean any of this code:', (', ').join(orpha_sugg.values) , '?\n')
            return []
        else : 
            orpha_hpo = df[df['ORPHA'] == clinical_data]['OMIM'].to_list()
            print('Parsing of', clinical_data, 'successful with result of :', len(orpha_hpo), ' OMIM code : ', (', ').join(orpha_hpo), '\n')
            return orpha_hpo
    
    else:
        print('The terminology is not recognized, please check the input format.')
        print('Allowable format is : ICD-10:xxxx OR ORPHA:xxxxx for clinical disorder', '\n')
        return []

In [96]:
# Convert OMIM code to OMIM Class Object
def omim_code2object(omim_set) :
    omim_object = []
    for item in list(set(omim_set)):
        try : 
            disease = Omim.get(int(item.strip('OMIM:')))
            omim_object.append(disease)
        except:
            print('OMIM code', item, 'is skipped.')
            continue
    
    return omim_object

In [97]:
# Serialized List of HPO code(s) to HPO Set Object
def hpo_code2set(hpo_set) :
    hpo_object = HPOSet.from_queries(list(set(hpo_set)))
    return hpo_object

In [98]:
# Get Similarity Check of OMIM with HPOSet provided
def omim_hpo_similarity(omim_set, hpo_set):
    print('Trying to get similarity check between OMIM and HPOSet')
    
    # Convert the OMIM to HPO Set Object
    print('Convert the OMIM code to HPO set object')
    omim_object = omim_code2object(omim_set)
    omim_diseases = [o.hpo_set() for o in omim_object]

    # Convert the HPO Code to HPO Set
    hpo_sets =  hpo_code2set(hpo_set)

    # Search prospective disease with similarity with HPO
    similarities = hpo_sets.similarity_scores(omim_diseases, method='graphic', combine='funSimMax')
    matrix = np.array(similarities)

    return matrix

In [99]:
# Automatic parsing
def phenotype_diagnosis_split(clinical_data_list) : 
    hpo_sets = []
    diagnosis_sets = []

    for clinical_data in clinical_data_list:
        print('Processing clinical data : ', clinical_data)
        
        # Case for SNOMEDCT if exist in HPO, then parse the HPO, else parse the ORPHA --> convert to OMIM
        if 'SNOMEDCT:' in clinical_data:
            snomed_hpo = hpo_parser(clinical_data, snomed2hpo_df)
            # If SNOMED is direct phenotype recognized by HPO
            if len(snomed_hpo) > 0:
                print('SNOMED-CT is recognized as clinical finding, parsing to HPO and add to list')
                hpo_sets.extend(snomed_hpo)
            else: # If SNOMED is clinical disorders, then convert to ORPHA --> convert to OMIM
                print('Trying to recognize SNOMED-CT as clinical disorder, parsing to ORPHA and respective OMIM format')
                snomed_orpha = snomed_orpha_parser(clinical_data, snomed2orpha_df)
                # Convert the ORPHA to OMIM
                for item in snomed_orpha:
                    orpha_omim = omim_parser(item, orpha2omim_df)
                    diagnosis_sets.extend(orpha_omim)

        # Case for ICD-10, lookup the OMIM directly
        elif 'ICD-10:' in clinical_data:
            icd_omim = omim_parser(clinical_data, icd10omim_df)
            diagnosis_sets.extend(icd_omim)
        
        # Case for ORPHA, lookup the OMIM directly
        elif 'ORPHA:' in clinical_data:
            orpha_omim = omim_parser(clinical_data, orpha2omim_df)
            diagnosis_sets.extend(orpha_omim)
        
        # Case for OMIM, directly extends the diagnosis_sets
        elif 'OMIM:' in clinical_data:
            diagnosis_sets.extend([clinical_data])

        # Case for LOINC, lookup the HPO directly
        elif 'LOINC:' in clinical_data:
            loinc_hpo = hpo_parser(clinical_data, loinc2hpo_df)
            hpo_sets.extend(loinc_hpo)
            
        # Case for HPO, directly extends the hpo_sets
        elif 'HP:' in clinical_data:
            hpo_sets.extend([clinical_data])

        else :
            print('The terminology is not recognized, please check the input format.')
            print('Allowable format is : ICD-10:xxxx OR ORPHA:xxxxx OR SNOMED-CT:xxxxx OR OMIM:xxxxx for clinical disorder to be converted to OMIM', '\n')
            print('Allowable format is : HP:xxxxx OR LOINC:xxxxx|Interpretation OR SNOMEDCT:xxxxx for clinical finding to be converted to HPO', '\n')
            continue


    return hpo_sets, diagnosis_sets

In [100]:
# Read line from clinical_data and parse the data to list
with open(clinical_data, 'r') as file:
    clinical_data_list = file.read().splitlines()

In [101]:
[hpo_sets, diagnosis_sets] = phenotype_diagnosis_split(clinical_data_list)

Processing clinical data :  SNOMEDCT:258211005
Trying to parse HPO from terminology SNOMEDCT:258211005
Parsing of SNOMEDCT:258211005 successful with result of : 1  HPO code :  HP:0000007 

SNOMED-CT is recognized as clinical finding, parsing to HPO and add to list
Processing clinical data :  SNOMEDCT:36760000
Trying to parse HPO from terminology SNOMEDCT:36760000
Parsing of SNOMEDCT:36760000 successful with result of : 1  HPO code :  HP:0001433 

SNOMED-CT is recognized as clinical finding, parsing to HPO and add to list
Processing clinical data :  SNOMEDCT:271737000
Trying to parse HPO from terminology SNOMEDCT:271737000
Parsing of SNOMEDCT:271737000 successful with result of : 1  HPO code :  HP:0001903 

SNOMED-CT is recognized as clinical finding, parsing to HPO and add to list
Processing clinical data :  SNOMEDCT:389026000
Trying to parse HPO from terminology SNOMEDCT:389026000
Parsing of SNOMEDCT:389026000 successful with result of : 1  HPO code :  HP:0001541 

SNOMED-CT is recogn

In [102]:
hpo_sets

['HP:0000007',
 'HP:0001433',
 'HP:0001903',
 'HP:0001541',
 'HP:0010972',
 'HP:0005561',
 'HP:0001396',
 'HP:0001410',
 'HP:0002910',
 'HP:0001410',
 'HP:0002910',
 'HP:0000938',
 'HP:0001642',
 'HP:0003073',
 'HP:0003233',
 'HP:0001873',
 'HP:0031964',
 'HP:0031956',
 'HP:0002366',
 'HP:0006568',
 'HP:0004333',
 'HP:0001531']